# Hello class!


Welcome to my individual project! 

### What is this thing?

This is an interactive "Flux based model" of ecoli metabolism. It is created using the [python](http://www.python.org) programming language running in a [jupyter notebook](http://jupyter.org/). 

### What is a Flux Based Model?

Organisms convert energy and food into useful molecules and proginy through series of many individual reactions. Estimating the kinetic parameters for each of these reactions under the varying conditions within in a cell is often impractical. Flux based models use stoicheometry of the reactions and some complicated math (linear optimization) to predict the concentrations of metabolites and the "flux" through reactions.

### How does it work

1. Input all of the known reactions of an organism into the model in terms of their stoicheometry.
2. Define an "objective function", for example cell growth rate.
3. Using linear optimization to see what reactions would be employed to maximize the objective function


## Getting started - Run the following cells!

The following code allows us to connect to a online database of metabolic information and build a stoicheometry matrix. Don't worry about how it works for the moment and just press run


In [205]:
## Run this cell so you are able to get reaction data from EcoCyc online database!

class BioCycAPI(object):
    def __init__(self):
        self.cache = {}

    def get_xml(self, identifier):
        if identifier in self.cache:
            return self.cache[identifier]
        
        resp = requests.get('https://websvc.biocyc.org/getxml?%s' % identifier)
        if resp.status_code != 200:
            raise Exception('Error fetching biocyc identifier "%s"' % identifier)
        result = fromstring(resp.text)
        self.cache[identifier] = result
        return result
        
    def get_compound(self, compound_identifier):        
        result = self.get_xml(compound_identifier)
        compound_xml = result.find('Compound')

        return {
            'common_name': compound_xml.find('common-name').text,
            'synonym': compound_xml.find('synonym').text,
            #'inchi': compound_xml.find('inchi').text
        }

    def parse_reaction_participant_xml(self, participant_xml):
        compound = participant_xml.find('Compound')
        compound_identifier = compound.attrib['resource'][7:]

        coefficient_xml = participant_xml.find('coefficient')
        if coefficient_xml is None:
            coefficient = 1
        else:
            coefficient = int(coefficient_xml.text)

        participant_details = api.get_compound(compound_identifier)
        participant_details['coefficient'] = coefficient
        return participant_details
    
    def get_reaction(self, reaction_identifier):
        result = self.get_xml(reaction_identifier)
        rxn_xml = result.findall('Reaction')[0]

        er_xml = rxn_xml.find('enzymatic-reaction')
        er_xml = er_xml.find('Enzymatic-Reaction')
        
        rxn_details = {
            'enzyme_ec_number': rxn_xml.find('ec-number').text,
            'enzyme_common_name': er_xml.find('common-name').text,
            'enzyme_synonym': er_xml.find('synonym').text
        }
        
        reactants = []
        for reactant_xml in rxn_xml.findall('left'):
            reactants.append(self.parse_reaction_participant_xml(reactant_xml))
        products = []
        for product_xml in rxn_xml.findall('right'):
            products.append(self.parse_reaction_participant_xml(product_xml))
        
        rxn_details['reactants'] = reactants
        rxn_details['products'] = products
        
        return rxn_details

api = BioCycAPI()

In [206]:
## NOW Run this cell so you can build Stoichiometry Matrices


import requests
from xml.etree.ElementTree import fromstring
from collections import OrderedDict
import itertools
import numpy as np

class StoichiometryStore(object):
    def __init__(self, biocyc_api):
        self.biocyc_api = biocyc_api
        
        self.reactions = OrderedDict()
        
    def add_reaction(self, name):
        if name in self.reactions:
            return
        
        self.reactions[name] = self.biocyc_api.get_reaction(name)
        
    def get_metabolites(self):
        metabolites = OrderedDict()
        for reaction in self.reactions.values():
            for participant in itertools.chain.from_iterable([
                reaction['products'], reaction['reactants']
            ]):
                
                metabolites[participant['common_name']] = {
                    'synonym': participant['synonym']
                }
                
        
        return metabolites
    
    def get_metabolite_vector(self):
        return list(self.get_metabolites().keys())
    
    def get_metabolite_positions(self):
        metabolite_list = self.get_metabolite_vector()
        return {
            metabolite_list[pos]: pos for pos in range(len(metabolite_list))
        }
    
    def get_reaction_vector(self):
        return list(self.reactions.keys())
    
    def get_stoichiometry_matrix(self):
        metabolite_positions = self.get_metabolite_positions()
        num_metabolites = len(metabolite_positions)
        num_reactions = len(self.reactions)
        
        
        
        s = np.zeros((num_metabolites, num_reactions))
        cur_rxn_idx = 0
        for reaction_name, reaction_details in self.reactions.items():
            for reactant_details in reaction_details['reactants']:
                # reactants should have negative coefficient
                coefficient = reactant_details['coefficient'] * -1
                s[metabolite_positions[reactant_details['common_name']], cur_rxn_idx] = coefficient

            for product_details in reaction_details['products']:
                # products should have positive coefficient
                coefficient = product_details['coefficient']
                s[metabolite_positions[product_details['common_name']], cur_rxn_idx] = coefficient

            
            cur_rxn_idx += 1
        
        return s

## An example to give you a feel for the technique

One pathway in ecoli metabolism is the creation of the amino acid threonine.

### online databases

There is so much data in molecular biology these days that researchers have created online databases to keep track of it. These databases contain the results of thousands of individual researchers efforts and their publications.

EcoCyc was created by researchers at SRI and stores metabolic pathways for Ecoli. It's available online at https://ecocyc.org/. Heres how the authors describe it:

    EcoCyc is a scientific database for the bacterium Escherichia coli K-12 MG1655. The EcoCyc project performs literature-based curation of the entire genome, and of transcriptional regulation, transporters, and metabolic pathways. 
    
    You can learn more by reading this paper: Keseler et al. (2017), "EcoCyc: reflecting new knowledge about Escherichia coli K-12", Nucleic Acids Research 45:D543-50.
    
    
EcoCyc has thousands of reactions and pathways for ecoli and you get visualize them on the web. Here is a link to the threonine synthesis pathway: https://ecocyc.org/ECOLI/NEW-IMAGE?type=NIL&object=THRESYN-PWY&redirect=T


![Threonine Biosynthesis Pathway](https://ecocyc.org/tmp/ptools-images/ECOLI/THRESYN-PWY_PWY-DIAGRAM_redirect=T.gif)



For this project we are interested in getting this data in a programatic way. Ecocyc can be accesssed by an "API". I've written some python code that lets us get information about each reaction of this pathway. For example, below we retrieve details about threonine synthase. The format is a bit hard to read, but easy to interact with using python.

In [203]:
# Retrieve information about the threonine synthase reaction
api.get_reaction('ECOLI:THRESYN-RXN')

{'enzyme_ec_number': 'EC-4.2.3.1',
 'enzyme_common_name': 'threonine synthase',
 'enzyme_synonym': 'O-phospho-L-homoserine phospho-lyase',
 'reactants': [{'common_name': 'O-phospho-L-homoserine',
   'synonym': 'o-phosphohomoserine',
   'coefficient': 1},
  {'common_name': 'H<sub>2</sub>O', 'synonym': 'H2O', 'coefficient': 1}],
 'products': [{'common_name': 'phosphate',
   'synonym': 'inorganic phosphate',
   'coefficient': 1},
  {'common_name': 'L-threonine', 'synonym': 'T', 'coefficient': 1}]}

In [1]:
stoich = StoichiometryStore(api)
stoich.add_reaction('ECOLI:THRESYN-RXN')

metabolites = stoich.get_metabolites()
print(stoich.get_metabolite_vector())
print(stoich.get_reaction_vector())
print(stoich.get_metabolite_positions())
print(stoich.get_stoichiometry_matrix())

import pprint
pprint.pprint(metabolites)
#api.get_compound('ECOLI:O-PHOSPHO-L-HOMOSERINE')

## References


1. W. Covert. Fundamentals of Systems Biology: From Synthetic Circuits to Whole-cell Models Markus. CRC Press. 2015.
2. Keseler et al. (2017), "EcoCyc: reflecting new knowledge about Escherichia coli K-12", Nucleic Acids Research 45:D543-50.

In [200]:
import requests
from xml.etree.ElementTree import fromstring
from collections import OrderedDict
import itertools
import numpy as np

class StoichiometryStore(object):
    def __init__(self, biocyc_api):
        self.biocyc_api = biocyc_api
        
        self.reactions = OrderedDict()
        
    def add_reaction(self, name):
        if name in self.reactions:
            return
        
        self.reactions[name] = self.biocyc_api.get_reaction(name)
        
    def get_metabolites(self):
        metabolites = OrderedDict()
        for reaction in self.reactions.values():
            for participant in itertools.chain.from_iterable([
                reaction['products'], reaction['reactants']
            ]):
                
                metabolites[participant['common_name']] = {
                    'synonym': participant['synonym']
                }
                
        
        return metabolites
    
    def get_metabolite_vector(self):
        return list(self.get_metabolites().keys())
    
    def get_metabolite_positions(self):
        metabolite_list = self.get_metabolite_vector()
        return {
            metabolite_list[pos]: pos for pos in range(len(metabolite_list))
        }
    
    def get_reaction_vector(self):
        return list(self.reactions.keys())
    
    def get_stoichiometry_matrix(self):
        metabolite_positions = self.get_metabolite_positions()
        num_metabolites = len(metabolite_positions)
        num_reactions = len(self.reactions)
        
        
        
        s = np.zeros((num_metabolites, num_reactions))
        cur_rxn_idx = 0
        for reaction_name, reaction_details in self.reactions.items():
            for reactant_details in reaction_details['reactants']:
                # reactants should have negative coefficient
                coefficient = reactant_details['coefficient'] * -1
                s[metabolite_positions[reactant_details['common_name']], cur_rxn_idx] = coefficient

            for product_details in reaction_details['products']:
                # products should have positive coefficient
                coefficient = product_details['coefficient']
                s[metabolite_positions[product_details['common_name']], cur_rxn_idx] = coefficient

            
            cur_rxn_idx += 1
        
        return s
            

stoich = StoichiometryStore(api)
stoich.add_reaction('ECOLI:THRESYN-RXN')

metabolites = stoich.get_metabolites()
print(stoich.get_metabolite_vector())
print(stoich.get_reaction_vector())
print(stoich.get_metabolite_positions())
print(stoich.get_stoichiometry_matrix())

import pprint
pprint.pprint(metabolites)
#api.get_compound('ECOLI:O-PHOSPHO-L-HOMOSERINE')

['phosphate', 'L-threonine', 'O-phospho-L-homoserine', 'H<sub>2</sub>O']
['ECOLI:THRESYN-RXN']
{'phosphate': 0, 'L-threonine': 1, 'O-phospho-L-homoserine': 2, 'H<sub>2</sub>O': 3}
[[ 1.]
 [ 1.]
 [-1.]
 [-1.]]
OrderedDict([('phosphate', {'synonym': 'inorganic phosphate'}),
             ('L-threonine', {'synonym': 'T'}),
             ('O-phospho-L-homoserine', {'synonym': 'o-phosphohomoserine'}),
             ('H<sub>2</sub>O', {'synonym': 'H2O'})])


In [165]:
rxn_xml = xml.findall('Reaction')[0]
reactants = []
for reactants_xml in rxn_xml.findall('left'):
    compound = reactants_xml.find('Compound')
    compound_identifier = compound.attrib['resource'][7:]
    
    coefficient_xml = product.find('coefficient')
    if coefficient_xml is None:
        coefficient = 1
    else:
        coefficient = int(coefficient_xml.text)
    
    reactant_details = api.get_compound(compound_identifier)
    reactant_details['coefficient'] = coefficient
    
print(reactants)

[]


In [88]:
er_xml = rxn_xml.find('enzymatic-reaction')
er_xml = er_xml.find('Enzymatic-Reaction')
er_xml.find('common-name').text
er_xml.find('synonym').text

'O-phospho-L-homoserine phospho-lyase'

In [158]:
x

{'enzyme_ec_number': 'EC-4.2.3.1',
 'enzyme_common_name': 'threonine synthase',
 'enzyme_synonym': 'O-phospho-L-homoserine phospho-lyase',
 'reactants': [{'common_name': 'O-phospho-L-homoserine',
   'synonym': 'o-phosphohomoserine',
   'coefficient': 2},
  {'common_name': 'H<sub>2</sub>O', 'synonym': 'H2O', 'coefficient': 2}]}

In [141]:
res = api.get_xml('ECOLI:RXN0-5268')

In [153]:
for product in res.find('Reaction').findall('left'):
    coefficient_xml = product.find('coefficient')
    if coefficient_xml is None:
        coefficient = 1
    else:
        coefficient = int(coefficient_xml.text)
    
    print(coefficient)

8
1
2
